In [1]:
%cd ..

C:\Users\georg\Documents\msc-project


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn import linear_model
from sklearn.decomposition import PCA
from src.evaluation import evaluate_model, compare_regression_models

In [3]:
df = pd.read_csv("data/interim/tracks.csv")

In [4]:
targets = MinMaxScaler(feature_range=(0,100)).fit_transform(np.array(df['views']).reshape(-1,1))
features = df[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','tempo','duration_ms','time_signature']]
data = StandardScaler().fit_transform(features)

In [5]:
compare_regression_models(data,targets)

Model                     R2     Ma     NM    
Baseline                  -0.022 -6.5e+01 -3.5e+01
Linear Regression         -0.17  -6.7e+01 -4.1e+01
Polynomial Regression     -4.4   -2.5e+02 -1.8e+02
Ridge Regression          -0.17  -6.7e+01 -4.1e+01
LARS Lasso                -0.022 -6.5e+01 -3.5e+01
Bayesian Ridge            -0.17  -6.7e+01 -4e+01
SGD                       -0.18  -6.7e+01 -4.1e+01
